In [1]:
!pip install kb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 11.3 MB/s eta 0:00:0031m11.7 MB/s eta 0:00:01


In [1]:
!pip install pyproj
!pip install -U numpy==1.26.4
!pip install transformers
!pip install -U accelerate
!pip install pandas
!pip install PyPDF2
!pip install NLTK
!pip install editdistance
!pip install  paramiko
!pip install func_timeout
!pip install kb

import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

  Obtaining dependency information for accelerate from https://files.pythonhosted.org/packages/73/de/64508cb91af013aaba214752309c0967568a4219d50a4ea30e822af3c976/accelerate-1.3.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.6/336.6 kB 8.8 MB/s eta 0:00:0010.8 MB/s eta 0:00:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.33.0
    Uninstalling accelerate-0.33.0:
      Successfully uninstalled accelerate-0.33.0


  Using cached func_timeout-4.3.5-py3-none-any.whl
  Obtaining dependency information for kb from https://files.pythonhosted.org/packages/bf/b8/377507327293936fe10f5c1e8993be9657b4c138fbc0c6466db0308a2988/kb-0.1.8-py3-none-any.whl.metadata


[nltk_data] Downloading package punkt to /home/alex/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/alex/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [15]:
# import libraries
import urllib.request
import requests
import json
from pyproj import Transformer
import os
import time
import sys
#from kb.nl.api import oai
#from kb.nl.helpers import alto_to_text
import csv

# import common functions
sys.path.insert(1, '../')
import common

# set up transformer from Lambert72 to lat/lon
transformer = Transformer.from_crs("EPSG:31370", "EPSG:4326")

# set name of module, to fetch info from config
module_name = "kb"

In [24]:
# get info from config file

config = common.get_config()
        
pdf_folder = config['data_source'][module_name]['pdf_folder']
print(f'pdf_folder: {pdf_folder}')

json_folder = config['data_source'][module_name]['json_folder']
print(f'json_folder: {json_folder}')

html_folder = config['data_source'][module_name]['html_folder']
print(f'html_folder: {html_folder}')

language = config['data_source'][module_name]['language']
print(f'language: {language}')

bert_model = config['bert_models'][language]
print(f'bert_model: {bert_model}')

document_metadata = config['data_source'][module_name]['document_metadata']
print(f'document_data: {document_metadata}')

# load metadata
metadata = {}
first = True
with open(document_metadata, mode ='r')as file:
    csvFile = csv.reader(file)
    for line in csvFile:
        if(first):
            first = False
        else:
            titleField = line[3]
            if titleField[:1] == '#':
                titleField = titleField[15:]
            titleField = titleField.replace('@','')
            splitTitleField = titleField.split(' / ')
            title = splitTitleField[0]
            if( len(splitTitleField) > 1):
                author = splitTitleField[1]
            else:
                author = ''
            metadata[line[0]] = {
                'year':line[2],
                'title':title,
                'author':author,
                'subtitle':line[4],
            }
print(metadata)

pdf_folder: /media/alex/Data/agnes_data/kb/pdf/
json_folder: /media/alex/Data/agnes_data/kb/json/
html_folder: /media/alex/Data/agnes_data/kb/html/
language: dutch
bert_model: /media/alex/Data/agnes_models/ArcheoBERTje-NER
document_data: /media/alex/Data/agnes_data/kb/document_metadata.csv
{'036167835': {'year': '1986', 'title': 'Op zoek naar mens en materiële cultuur : feestbundel aangeboden aan J.D. van der Waals ter gelegenheid van zijn emeritaat', 'author': 'onder red. van Harry Fokkens, Pieteke Banga en Mette Bierma', 'subtitle': ''}, '036210242': {'year': '1987', 'title': 'Opgegraven beschaving : de geschiedenis van de beschaving aan de hand van archeologische vondsten', 'author': 'Ruth Whitehouse en John Wilkins ; [vert. uit het Engels door M.W. Blok]', 'subtitle': ''}, '036295191': {'year': '1985', 'title': 'Archeologische kroniek van Overijssel over 1982-1984', 'author': 'A.D. Verlinde', 'subtitle': ''}, '036295590': {'year': '1985', 'title': 'Archeologische kroniek van Holla

865192715 /media/alex/Data/agnes_data/kb/pdf/865192715.pdf
{'year': '1986', 'title': 'Verleden land : archeologische opgravingen in Nederland', 'author': 'J.H.F. Bloemers, L.P. Louwe Kooijmans [en] H. Sarfatij ; [red. Annemiek Overbeek ; fotogr. Henk Brandsen ; ill. Bob Brobbel ; krt. Armand Haye]', 'subtitle': ''}
054861527 /media/alex/Data/agnes_data/kb/pdf/054861527.pdf
{'year': '1989', 'title': 'Archeologische kroniek van Overijssel over 1987', 'author': 'A.D. Verlinde', 'subtitle': ''}
862873622 /media/alex/Data/agnes_data/kb/pdf/862873622.pdf
{'year': '1986', 'title': 'Vondsten uit het verleden : oudheidkundig bodemonderzoek : archeologisch jaarboek ...', 'author': 'uitg. in opdracht van de Stichting voor de Nederlandse Archeologie Amersfoort', 'subtitle': ''}
832399159 /media/alex/Data/agnes_data/kb/pdf/832399159.pdf
{'year': '1981', 'title': 'Archeologie in Tilburg en omgeving', 'author': 'verslag van het Oudheidkundig Centrum "De Oliemeulen"', 'subtitle': ''}
841548102 /media/

In [31]:



# go through folder of PDFs
for file in os.listdir(pdf_folder):
    
    print(f"Processing PPN:{ppn}")

    pdf_file = os.path.join(pdf_folder,file)
    ppn = file[:-4]
    meta = metadata[ppn]
    uri = f'https://webggc.oclc.org/cbs/DB=2.37/CMD?ACT=SRCHM&MATCFILTER=Y&MATCSET=Y&NOSCAN=Y&PARSE_MNEMONICS=N&PARSE_OPWORDS=N&PARSE_OLDSETS=N&IMPLAND=Y&ACT0=SRCHA&screen_mode=zoeken&IKT0=6120&TRM0={ppn}&ACT1=*&IKT1=4&TRM1=&ACT2=*&IKT2=1004&TRM2=&ACT3=*&IKT3=8050&TRM3=&SRT=LST_Ya&ADI_TAA=&ADI_JVU='

    output_document = {}
    output_document['source'] = module_name
    output_document['file_name'] = file
    output_document['file_type'] = 'book'
    output_document['title'] = meta['title']
    output_document['creators'] = meta['author']
    output_document['description'] = '' # no descriptions in this data source
    output_document['createdAt'] = meta['year']
    output_document['identifiers'] = {'ppn':ppn,'uri':uri}
    output_document['language'] = 'Dutch'

    #print(json.dumps(output_document, indent=4))

    # save document.json 
    json_output_folder = f"{json_folder}/{ppn}"
    common.savejson(output_document, f"{json_output_folder}/document.json")

    print(f"saved doc json")

    # process pdf, store page.json files with entities 
    common.run_ner_on_pdf(
        pdf_file, 
        json_output_folder, 
        bert_model, 
        language
    )

    print(f"ran NER, saved page json")

    # process pdf, save html files
    html_output_folder = f"{html_folder}/{ppn}"
    common.pdf2html(pdf_file, html_output_folder)

    print(f"generated and saved html")

                




Processing PPN:865192715
saved doc json
ran NER, saved page json
Converted /media/alex/Data/agnes_data/kb/pdf/865192715.pdf to html!
generated and saved html
Processing PPN:865192715
saved doc json
ran NER, saved page json
Converted /media/alex/Data/agnes_data/kb/pdf/054861527.pdf to html!
generated and saved html
Processing PPN:054861527
saved doc json
ran NER, saved page json
Converted /media/alex/Data/agnes_data/kb/pdf/862873622.pdf to html!
generated and saved html
Processing PPN:862873622
saved doc json
ran NER, saved page json
Converted /media/alex/Data/agnes_data/kb/pdf/832399159.pdf to html!
generated and saved html
Processing PPN:832399159
saved doc json
ran NER, saved page json
Converted /media/alex/Data/agnes_data/kb/pdf/841548102.pdf to html!
generated and saved html
Processing PPN:841548102
saved doc json
ran NER, saved page json
Converted /media/alex/Data/agnes_data/kb/pdf/853924171.pdf to html!
generated and saved html
Processing PPN:853924171
saved doc json
ran NER, sav

In [34]:
# upload json and html to webserver
common.upload2webserver(json_folder, html_folder, module_name, config['webserver']['json_folder'], config['webserver']['html_folder'])

print(f"uploaded json/html to webserver")

# remotely start indexing script on webserver
common.start_index(module_name)

print(f"indexing on webserver started")

print(f"done!")

uploaded json/html to webserver
indexing on webserver started
done!
